<a href="https://www.kaggle.com/code/mahammadashad/mental-health-analysis-model?scriptVersionId=205598928" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

## Mental Health Classification using Bert and Deepspeed <br>

### In this model i have two parts 
#### 1. Untrained model without using Deepspeed 
#### 2. train.py file with traning script for Deepspeed training  <br>

#### Why i have two parts , its because i have all the preprocessing visualization and other components explained in this notebook so it's easier to understand.
#### In the train.py file i have not explained much so i include the first part. 

#####  Hope this will be of some use , Thank You

In [1]:

import pandas as pd
import re
import os
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer
import torch
from torch.utils.data import DataLoader
from transformers import BertModel,BertTokenizerFast,BertForSequenceClassification, Adafactor
from tqdm import tqdm

In [2]:
df = pd.read_csv("/kaggle/input/sentiment-analysis-for-mental-health/Combined Data.csv")

In [3]:
df

,Unnamed: 0,statement,status
0,0,oh my gosh,Anxiety
1,1,"trouble sleeping, confused mind, restless hear...",Anxiety
2,2,"All wrong, back off dear, forward doubt. Stay ...",Anxiety
3,3,I've shifted my focus to something else but I'...,Anxiety
4,4,"I'm restless and restless, it's been a month n...",Anxiety
...,...,...,...
53038,53038,Nobody takes me seriously I’ve (24M) dealt wit...,Anxiety
53039,53039,"selfishness ""I don't feel very good, it's lik...",Anxiety
53040,53040,Is there any way to sleep better? I can't slee...,Anxiety
53041,53041,"Public speaking tips? Hi, all. I have to give ...",Anxiety


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53043 entries, 0 to 53042
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  53043 non-null  int64 
 1   statement   52681 non-null  object
 2   status      53043 non-null  object
dtypes: int64(1), object(2)
memory usage: 1.2+ MB


In [5]:
# Displaying the full length
pd.set_option('display.max_colwidth', None)
df.head()


,Unnamed: 0,statement,status
0,0,oh my gosh,Anxiety
1,1,"trouble sleeping, confused mind, restless heart. All out of tune",Anxiety
2,2,"All wrong, back off dear, forward doubt. Stay in a restless and restless place",Anxiety
3,3,I've shifted my focus to something else but I'm still worried,Anxiety
4,4,"I'm restless and restless, it's been a month now, boy. What do you mean?",Anxiety


# Checking for null values

In [6]:
df.isna().sum()

Unnamed: 0      0
statement     362
status          0
dtype: int64

In [7]:
#Removing Null Values
df.dropna(inplace=True)

In [8]:
df.isna().sum()

Unnamed: 0    0
statement     0
status        0
dtype: int64

In [9]:
# Converting 'statement' to column to string for using it in BERTokenizer

df["str_statement"]= df["statement"].astype(str)

In [10]:
df["str_statement"].head()

0                                                                        oh my gosh
1                  trouble sleeping, confused mind, restless heart. All out of tune
2    All wrong, back off dear, forward doubt. Stay in a restless and restless place
3                     I've shifted my focus to something else but I'm still worried
4          I'm restless and restless, it's been a month now, boy. What do you mean?
Name: str_statement, dtype: object

# Covert to lowercase

In [11]:
df['str_statement'] = df['statement'].str.lower()

In [12]:
df["str_statement"].head()

0                                                                        oh my gosh
1                  trouble sleeping, confused mind, restless heart. all out of tune
2    all wrong, back off dear, forward doubt. stay in a restless and restless place
3                     i've shifted my focus to something else but i'm still worried
4          i'm restless and restless, it's been a month now, boy. what do you mean?
Name: str_statement, dtype: object

# Remove links and unwanted things like username,links etc

In [13]:
def remove_patterns(text):
    # URL Removal
    text = re.sub(r'http[s]?://\S+','',text)
    # Remove markdown style links
    text = re.sub(r'\[.*?\]\(.*?\)', '', text)
    # Remove handles (that start with '@')
    text = re.sub(r'@\w+', '', text)
    # Remove punctuation and other special characters
    text = re.sub(r'[^\w\s]', '', text)
    return text.strip()
    
df['str_statement'] = df['str_statement'].apply(remove_patterns)
df.head()

,Unnamed: 0,statement,status,str_statement
0,0,oh my gosh,Anxiety,oh my gosh
1,1,"trouble sleeping, confused mind, restless heart. All out of tune",Anxiety,trouble sleeping confused mind restless heart all out of tune
2,2,"All wrong, back off dear, forward doubt. Stay in a restless and restless place",Anxiety,all wrong back off dear forward doubt stay in a restless and restless place
3,3,I've shifted my focus to something else but I'm still worried,Anxiety,ive shifted my focus to something else but im still worried
4,4,"I'm restless and restless, it's been a month now, boy. What do you mean?",Anxiety,im restless and restless its been a month now boy what do you mean


# Convert the labels (strings) to integers using Label Encoder

In [14]:
label_encoder = LabelEncoder()
df['encoded_status'] = label_encoder.fit_transform(df['status'])

# Splitting the data and Tokenzation using BertTokenizerFast

In [15]:
# Initialize the tokenizer for the pre-trained BERT model
tokenizer = BertTokenizerFast.from_pretrained("bert-base-uncased")

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


## Split the dataset into training and validation sets

In [16]:
train_texts, val_texts, train_labels, val_labels = train_test_split(
    df['str_statement'],  
    df['encoded_status'],
    test_size=0.25, # Here 25 percent of the dat will be used for validation
    random_state=42
)

In [17]:
# Tokenization using BERTTokenizerFAst

max_length= 512
train_encodings= tokenizer(train_texts.tolist(),padding='max_length',truncation=True,return_tensors='pt') # Here it will return the tokens as tensors
val_encodings =tokenizer(val_texts.tolist(),padding='max_length',truncation=True,return_tensors='pt')

# Convert the tokenized encodings to a PyTorch dataset

In [18]:
import torch

class MentalHealthDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = torch.tensor(labels)

    def __getitem__(self, idx):
        item = {key: val[idx].clone().detach() for key, val in self.encodings.items()}
        item['labels'] = self.labels[idx].clone().detach()

        return item

    def __len__(self):
        return len(self.labels)

# Create the datasets
train_dataset = MentalHealthDataset(train_encodings, train_labels.tolist()) # You need to convert it to list 
val_dataset = MentalHealthDataset(val_encodings, val_labels.tolist()) # You need to convert it to list 

In [19]:
from torch.utils.data import DataLoader

# Create DataLoader for both training and validation datasets
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True,num_workers=2) # You can change batch size as per your need
val_loader = DataLoader(val_dataset,  batch_size=32)

Initializing 

In [20]:
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=7) 

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [21]:
# Move model to GPU (if available)
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

## You are free to use whatever type of trainers you want after this part for your purpose

<bR> --------------------------------------------------------------------------------------

# Deepspeed Part

In [22]:
!pip install deepspeed
import deepspeed

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 22.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - \ | / done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.0/54.0 kB 2.8 MB/s eta 0:00:00
  Created wheel for deepspeed: filename=deepspeed-0.15.3-py3-none-any.whl size=1526207 sha256=fc5896da71aaf35de0248cc54238d5d34599c036f19e42f506e5990890fc5c7d
  Stored in directory: /root/.cache/pip/wheels/b3/c2/9f/37a2c813b8d64d7908793319cfdfa4f852754e177f20f0b858
Successfully built deepspeed
[2024-11-06 14:53:37,895] [INFO] [real_accelerator.py:219:get_accelerator] Setting ds_accelerator to cuda (auto detect)


/opt/conda/compiler_compat/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status
/opt/conda/compiler_compat/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status


## You need mpi4y to run deepspeed

In [23]:
!conda install -y mpi4py

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Retrieving notices: ...working... done
Channels:
 - rapidsai
 - nvidia
 - nodefaults
 - conda-forge
 - defaults
 - pytorch
Platform: linux-64
Solving environment: \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / done

## Package Plan ##

  environment location: /opt/conda

  added / updated specs:
    - mpi4py


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    conda-24.9.2               |  py310hff52083_0         895 KB  conda-forge
    mpi-1.0.1                  |            mpich           6 KB  conda-forge
    mpi4py-4.0.1               |  py310hc9139ab_0         802 KB  conda-forge
    mpich-4.2.3                |     h670b19f_100        13.1 MB  conda-forge
    ------------------------------------------------------------
                                           Total:        1

# To run train the model using Deepseed while utilizing multiple gpu 
##### (kaggle does not have native multi-gpu processing(CLI) support.<br>


#### **1. Create a .py file with all the preprocessing and working of the model inside a main function using %%write(as below)**
(You can convert a notebook file to .py by going to the file section and selcting script in editor type)

#### 2. add  if ＿name＿ == '＿main＿': main() at the end of the file 

#### 3. Run using accelerate launch


In [24]:
%%writefile train.py

def main():
    
    # Import necessary packages
    import pandas as pd
    import re
    from sklearn.preprocessing import LabelEncoder
    from sklearn.model_selection import train_test_split
    from transformers import BertTokenizerFast, BertForSequenceClassification
    import torch
    from torch import nn
    from torch.utils.data import DataLoader
    from transformers import Adafactor
    from tqdm import tqdm
    import deepspeed
    import json
    from accelerate import Accelerator
    from torch.optim import AdamW
    from transformers import get_linear_schedule_with_warmup
    from sklearn.metrics import f1_score, classification_report

    # Load dataset
    df = pd.read_csv("/kaggle/input/sentiment-analysis-for-mental-health/Combined Data.csv")

    # Convert 'statement' column to string
    df["str_statement"] = df["statement"].astype(str)
    # Ensure all values in 'str_statement' are strings
    df["str_statement"] = df["str_statement"].astype(str)

    # Convert to lowercase
    df['str_statement'] = df['str_statement'].str.lower()
    
    def remove_patterns(text):
        if isinstance(text, str):
            # URL Removal
            text = re.sub(r'http[s]?://\S+','',text)
            # Remove markdown style links
            text = re.sub(r'\[.*?\]\(.*?\)', '', text)
            # Remove handles (that start with '@')
            text = re.sub(r'@\w+', '', text)
            # Remove punctuation and other special characters
            text = re.sub(r'[^\w\s]', '', text)
            return text.strip()
    
    df['str_statement'] = df['str_statement'].apply(remove_patterns)

    df.dropna(inplace=True)
    
    # Label encoding
    label_encoder = LabelEncoder()
    df['encoded_status'] = label_encoder.fit_transform(df['status'])
    
    # Initialize the tokenizer for BERT
    tokenizer = BertTokenizerFast.from_pretrained("bert-base-uncased")

    # Split data into training and validation sets
    train_texts, val_texts, train_labels, val_labels = train_test_split(
        df['str_statement'], df['encoded_status'], test_size=0.25, random_state=42
    )

    # Tokenization
    max_length = 512
    train_encodings = tokenizer(train_texts.tolist(), padding='max_length', truncation=True, return_tensors='pt')
    val_encodings = tokenizer(val_texts.tolist(), padding='max_length', truncation=True, return_tensors='pt')

    # PyTorch Dataset Class
    class MentalHealthDataset(torch.utils.data.Dataset):
        def __init__(self, encodings, labels):
            self.encodings = encodings
            self.labels = torch.tensor(labels)

        def __getitem__(self, idx):
            item = {key: val[idx].clone().detach() for key, val in self.encodings.items()}
            item['labels'] = self.labels[idx].clone().detach()
            return item

        def __len__(self):
            return len(self.labels)

    # Create datasets
    train_dataset = MentalHealthDataset(train_encodings, train_labels.tolist())
    val_dataset = MentalHealthDataset(val_encodings, val_labels.tolist())

    # Create DataLoader for both training and validation datasets
    train_loader = DataLoader(train_dataset, batch_size=54, shuffle=True,pin_memory=True)
    val_loader = DataLoader(val_dataset, batch_size=54)

    # Define the BERT model
    model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=7)

    # Move model to GPU (if available)
    device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
    model.to(device)

    ds_config ="""
    {
    "train_batch_size": 54,
    "gradient_accumulation_steps": 1,
    "fp16": {
        "enabled": true,
        "min_loss_scale":1
    },
    "zero_optimization": {
        "stage": 2,
        "allgather_partitions": true,
        "reduce_scatter": true,
        "overlap_comm": true,
        "cpu_offload": true
    },
    "autotuning": {
        "enabled": true,
        "arg_mappings": {
            "train_micro_batch_size_per_gpu": "--per_device_train_batch_size",
            "gradient_accumulation_steps": "--gradient_accumulation_steps"
        }
        },
    "gradient_clipping": 1.0,
    "results_dir": "./autotuning_results",
    "exps_dir": "./autotuning_exps",
    "zero_allow_untested_optimizer": true,
    "steps_per_print": 2000,
    "wall_clock_breakdown": false,
    "scheduler": {
    "type": "WarmupLR",
    "params": {
      "warmup_min_lr": 0,
      "warmup_max_lr": 0.001,
      "warmup_num_steps": 1000
      }
      }
    "optimizer": {
           "type": "AdamW",
           "params": {
               "lr": 1e-4,
               "betas": [0.9, 0.999],
               "eps": 1e-8,
               "weight_decay": 0.01
        }   
      }
    }

    """
    # DeepSpeed Initialization
    with open("ds_config.json", "w") as f:
        f.write(ds_config)
   
    #optimizer = AdamW(model.parameters(), lr=0.001, betas=(0.9, 0.999), eps=1e-08, weight_decay=0.01)
    model_engine,optimizer, _, _ = deepspeed.initialize(
    model=model,
    config="ds_config.json" )
    
    model = model.to(device)
    
    accelerator = Accelerator()
    
    # Training loop
    from tqdm import tqdm
    from collections import OrderedDict
    
    criterion = nn.CrossEntropyLoss()
    epochs = 3
    num_batches = len(train_loader)
    total_steps = (num_batches) * (epochs)


    def train_one_epoch(model, train_loader,criterion,optimizer,accelarator, device):
        model.train()  # Set model to training mode
        total_loss = 0

        # Use tqdm for progress bar
        with tqdm(train_loader, leave=True, disable=not accelerator.is_local_main_process) as pbar:
            for idx, batch in enumerate(pbar):
                batch = {k: v.to(device) for k, v in batch.items()}  # Move batch to device

                outputs = model(**batch)
                loss = outputs.loss  # Assuming outputs has a loss attribute
            
                # Backward pass with accelerator
                model_engine.backward(loss)  # Automatic handling of backward pass
            
                # Update the loss and progress bar
                total_loss += loss.item()

                # Step the optimizer
                optimizer.step()
                optimizer.zero_grad()  # Clear gradients
            
                # Update the progress bar
                pbar.update(1)  
            

        # Wait for all processes to finish
        accelerator.wait_for_everyone()
        # Log average loss
        accelerator.print(f'train_loss: {total_loss / len(train_loader):.6f}')

    # Training loop
    for epoch in range(epochs):
        train_one_epoch(model, train_loader,criterion, optimizer, accelerator, device)
        print(f"Epoch {epoch + 1} completed.")


    torch.cuda.empty_cache() # Clear CUDA memory
    
    # Move model to GPU (if available)
    device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
    model.to(device)

    model.eval()
    if torch.cuda.device_count() > 1:
        model = torch.nn.DataParallel(model)

    
    val_loader = DataLoader(val_dataset, batch_size=128)

    # Store predictions and true labels
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for batch in tqdm(val_loader, desc="Making Predictions", unit="batch"):
            # Move the batch to the GPU (if available)
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            # Forward pass
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            logits = outputs.logits

            # Get the predicted labels
            preds = torch.argmax(logits, dim=1).cpu().numpy()
            labels = labels.cpu().numpy()

            # Store predictions and true labels
            all_preds.extend(preds)
            all_labels.extend(labels)

    # Calculate the F1-score
    f1 = f1_score(all_labels, all_preds, average='weighted')
    print(f'Validation F1-score: {f1:.4f}')


    # Save the trained model
    model.save_pretrained("./trained_model")
    
if __name__ == '__main__':
    main()


Writing train.py


In [25]:
import accelerate

## Start training using accelerate launch

In [26]:
os.environ["TOKENIZERS_PARALLELISM"] = "false" # To avoid warnings

In [27]:
!accelerate launch train.py --precision fp16 # using fp16 mixed-precision

[2024-11-06 14:55:27,064] [INFO] [real_accelerator.py:219:get_accelerator] Setting ds_accelerator to cuda (auto detect)
[2024-11-06 14:55:27,069] [INFO] [real_accelerator.py:219:get_accelerator] Setting ds_accelerator to cuda (auto detect)
/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/3

# Thank You ( Do upvote if you like this notebook)